<a href="https://colab.research.google.com/github/22Ifeoma22/22Ifeoma22/blob/main/05_GradientTape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.tensorflow.org/guide/autodiff

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

### Computing gradients

To differentiate automatically, TensorFlow needs to remember what operations happen in what order during the forward pass. Then, during the backward pass, TensorFlow traverses this list of operations in reverse order to compute gradients.

In [2]:
x = tf.Variable(4.0)

with tf.GradientTape() as tape:
    y = x**2

In [3]:
y

<tf.Tensor: shape=(), dtype=float32, numpy=16.0>

In [4]:
dy_dx = tape.gradient(y, x)

dy_dx

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [5]:
w = tf.Variable(tf.random.normal((4, 2)))

w

<tf.Variable 'Variable:0' shape=(4, 2) dtype=float32, numpy=
array([[ 1.6781247 ,  1.1388843 ],
       [ 0.22586143, -2.206287  ],
       [-0.32982063,  1.4358617 ],
       [-0.37560588,  1.3389487 ]], dtype=float32)>

In [6]:
b = tf.Variable(tf.ones(2, dtype=tf.float32))

b

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>

In [7]:
x = tf.Variable([[10., 20., 30., 40.]], dtype=tf.float32)

x

<tf.Variable 'Variable:0' shape=(1, 4) dtype=float32, numpy=array([[10., 20., 30., 40.]], dtype=float32)>

In [10]:
with tf.GradientTape(persistent=True) as tape:
    y = tf.matmul(x, w) + b

    loss = tf.reduce_mean(y**2)

In [11]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

The gradient with respect to each source has the shape of the source

In [12]:
dl_dw

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[ -26.203785,  648.96906 ],
       [ -52.40757 , 1297.9381  ],
       [ -78.61136 , 1946.9071  ],
       [-104.81514 , 2595.8762  ]], dtype=float32)>

In [13]:
dl_db

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.6203785, 64.896904 ], dtype=float32)>

In [24]:
layer = tf.keras.layers.Dense(2, activation='relu')

x = tf.constant([[10., 20., 30.]])

In [15]:
with tf.GradientTape() as tape:
    y = layer(x)

    loss = tf.reduce_sum(y**2)

grad = tape.gradient(loss, layer.trainable_variables)

In [16]:
grad

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 601.47205,    0.     ],
        [1202.9441 ,    0.     ],
        [1804.4161 ,    0.     ]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([60.147205,  0.      ], dtype=float32)>]

In [17]:
for var, g in zip(layer.trainable_variables, grad):
    print(f'{var.name}, shape: {g.shape}')

dense/kernel:0, shape: (3, 2)
dense/bias:0, shape: (2,)


### Gradients are calculated only with respect to trainable variables

Trainable variable, the value associated with this will be updated during the training process

In [25]:
x1 = tf.Variable(5.0)

x1

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

Trainable has been explicitly set to false

In [26]:
x2 = tf.Variable(5.0, trainable=False)

x2

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

Tensor, not a variable. Gradients are not calculated on Tensors

In [27]:
x3 = tf.add(x1, x2)

x3

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [28]:
x4 = tf.constant(5.0)

x4

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

In [29]:
with tf.GradientTape() as tape:
    y = (x1**2) + (x2**2) + (x3**2) + (x4**2)

grad = tape.gradient(y, [x1, x2, x3, x4])

grad

[<tf.Tensor: shape=(), dtype=float32, numpy=10.0>, None, None, None]

### Watch constants to calculate gradients with respect to them

tf.GradientTape provides hooks that give the user control over what is or is not watched. To record gradients with respect to a tf.Tensor, you need to call GradientTape.watch(x)

In [30]:
x1 = tf.constant(5.0)

x2 = tf.Variable(3.0)

In [31]:
with tf.GradientTape() as tape:
    tape.watch(x1)

    y = (x1**2) + (x2**2)

In [32]:
[dy_dx1, dy_dx2] = tape.gradient(y, [x1, x2])

dy_dx1, dy_dx2

(<tf.Tensor: shape=(), dtype=float32, numpy=10.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=6.0>)

In [33]:
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x1)

    y = (x1**2) + (x2**2)

In [34]:
[dy_dx1, dy_dx2] = tape.gradient(y, [x1, x2])

dy_dx1, dy_dx2

(<tf.Tensor: shape=(), dtype=float32, numpy=10.0>, None)

### Gradient tape records operations as they occur

Conditionals are naturally handled. The gradient only connects to the variable that was used.

In [35]:
x = tf.constant(1.0)
x1 = tf.Variable(5.0)
x2 = tf.Variable(3.0)

In [36]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)

    if x > 0.0:
        result = x1**2
    else:
        result = x2**2

dx1, dx2 = tape.gradient(result, [x1, x2])

dx1, dx2

(<tf.Tensor: shape=(), dtype=float32, numpy=10.0>, None)

In [37]:
x = tf.constant(-1.0)
x1 = tf.Variable(5.0)
x2 = tf.Variable(3.0)

In [38]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)

    if x > 0.0:
        result = x1**2
    else:
        result = x2**2

dx1, dx2 = tape.gradient(result, [x1, x2])

dx1, dx2

(None, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>)

In [39]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
    z = y * y

    dy_dx = tape.gradient(z, x)

print(dy_dx)

None
